In [1]:
 %load_ext autoreload

In [2]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets,transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import sys
import glob
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from visdom import Visdom

/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

# Function

In [3]:
path_folder = '/home/etienne/Dropbox/experiments'
path_tensorboard = '/home/etienne/Dropbox/Tensorboard'

In [4]:
def load_model(expname) :
    print('Loading : {}'.format(expname))
    pattern = '(?<=epoch_)(\d+)'
    natural_keys = lambda s: int(re.search(pattern, s).group(0))
    files = glob.glob(os.path.join(path_folder,expname,'models')+'/*.tar')
    files.sort(key=natural_keys)
    filename = files[-1]
    checkpoint = torch.load(os.path.join(path_folder,expname,'models',filename))
    print(os.path.join(path_folder,expname,'sample'))
    sys.path.insert(0,os.path.join(path_folder,expname,'sample'))
    try : 
        from nets.SPN import SPN
        net = SPN()
        net.load_state_dict(checkpoint['model'])
    except Exception as e :
        print("Didn't load {} : {}".format(expname,e))
        pass
    #sys.path.remove(os.path.join(path_folder,expname,'code','sample'))
    return net, filename

In [21]:
def create_tensorboard(expname,net) :
    writer = SummaryWriter(os.path.join(path_tensorboard,expname))
    writer.add_graph(net,torch.rand(1,3,1,1).to('cuda:0'))
    for name,p in net.named_parameters() :
        n = name.split('.')
        if len(n) == 3 :
            writer.add_histogram(n[0]+'_'+n[1]+'/'+n[2],p)
        if len(n) == 2 :
             writer.add_histogram(n[0]+'/'+n[1],p)

In [6]:
def save_model_pt(net,name) :
    '''
    Take the loaded model and save it as a .pt file in the 
    '''
    torch.save(net,name.replace('.tar','.pt'))

In [8]:
sys.path.insert(0,'/home/etienne/Dropbox/experiments/train_Resnet34_256_RetinaDatasetClassif_Freeze_48_bn_dr/sample')

In [9]:
m = torch.load('/home/etienne/Dropbox/experiments/train_Resnet34_256_RetinaDatasetClassif_Freeze_48_bn_dr/models/train_Resnet34_256_RetinaDatasetClassif_Freeze_48_bn_dr_full_model.pt')

In [24]:
m.classifier

Sequential(
  (dr1): Dropout(p=0.5)
  (d1): Linear(in_features=10752, out_features=2048, bias=True)
  (bn1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (dr2): Dropout(p=0.5)
  (d2): Linear(in_features=2048, out_features=1, bias=True)
)

In [22]:
create_tensorboard('train_Resnet34_256_RetinaDatasetClassif_Freeze_48_bn_dr',m)

/usr/local/lib/python3.6/dist-packages/torch/onnx/symbolic.py:173: UserWarning: ONNX export failed on adaptive_max_pool2d because output size that are not factor of input size not supported
  warnings.warn("ONNX export failed on " + op + " because " + msg + " not supported")


# Launch

In [7]:
@interact_manual
def load_experiments(exp=os.listdir(path_folder)):
    global net
    expname = exp
    print(expname)
    net, filename = load_model(expname)
    save_model_pt(net,os.path.join(path_folder,expname,'models',filename))
    create_tensorboard(expname,net)

interactive(children=(Dropdown(description='exp', options=('train_resnet34_full_size_mfreeze_108_pretrained_cr…

# Save all model pt at once

In [7]:
for expname in os.listdir(path_folder) :
    %autoreload
    try :
        net, filename = load_model(expname)
        save_model_pt(net,os.path.join(path_folder,expname,'models',filename))
    except Exception as e :
        print('Failed')

Loading : test_2_Sigmoid
Failed
Loading : train_2_SPN_vgg16
Loading : train_3_old_SPN_3_pooling
Loading : test_SPN_8
Failed
Loading : test_loss
Failed
Loading : mini_test_SPN_1
Loading : mini_test_Resnet_d1
Failed
Loading : train_1_resnet_34_freeze
Loading : test_12
Loading : train_3_old_SPN_3_pooling_256_size
Loading : te2
Failed
Loading : test_Sigmoid


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'SPN.SPN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Loading : train_1_SPN_3_pooling_32_size_unfreeze_dropout1
Loading : test_BCEwithlogits
Failed
Loading : test_tensorboard_1
Failed
Loading : train_3_old_SPN_3_pooling_128_size
Loading : test_blade
Loading : train_SPN_vgg16_1_mi_freeze
Loading : test_plotly_2
Loading : experimentation_vgg16_size32_2
Failed
Loading : experimentation_resnet34_size32
Loading : train_1_Resnet50_256_size_pretrained_freeze
Loading : mini_test_dumbnet_1
Failed
Loading : train_3_old_SPN_3_pooling_32_size
Loading : train_1_SPN_3_pooling_256_size_unfreeze_batch_norm
Loading : mini_test_resnet_2
Loading : test_tensorboard_3
Loading : test_dumb_8
Loading : pos_weight_test
Loading : mini_test_SPP_d4
Loading : test_features_8
Failed
Loading : experimentation_vgg16_size32
Loading : experimentation_vgg16_size32_3
Loading : train_1_SPN_3_pooling_32_size
Loading : train_1_SPN_3_pooling_256_size_CLR
Failed
Loading : train_1_SPN_3_pooling_256_size
Loading : SharkDetector_test_Histogram
Loading : train_1_old_SPN_vgg16
Loadin